In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200708']:
    print('----------------------------------------------------------------')
    print(y)
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_52_03_day_pcap\\mdL2Pcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSH1["StockID"] = logSH1["ID"] - 1000000
    logSH1 = logSH1[["sequenceNo", "StockID", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q", 
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", 
                     "ask2q", "ask3q", "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSH1 = logSH1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
        
    readPath = 'A:\\KR_daily_data\\' + y + '\\SH\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    logSH2 = []
    ll = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        logSH2 += [df]
    del df
    logSH2 = pd.concat(logSH2).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    for i in range(1, 6):
        if i == 1:
            logSH2["bid" + str(i) + "p"] = logSH2["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSH2["ask" + str(i) + "p"] = logSH2["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSH2["bid" + str(i) + "q"] = logSH2["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            logSH2["ask" + str(i) + "q"] = logSH2["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            logSH2["bid" + str(i) + "p"] = logSH2["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSH2["ask" + str(i) + "p"] = logSH2["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSH2["bid" + str(i) + "q"] = logSH2["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            logSH2["ask" + str(i) + "q"] = logSH2["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    logSH2 = logSH2.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    logSH2["time"] = (logSH2["QuotTime"] - int(y)*1000000000).astype(np.int64)
    
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time", "numTrades"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
    
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    for cols in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p","ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["IOPV"].count()
    n2 = test["sequenceNo"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo"])])
        display(len(test[np.isnan(test["sequenceNo"])])/n1)
        display(len(test[np.isnan(test["sequenceNo"])]["time"].unique()))
        display(test[np.isnan(test["sequenceNo"])]["time"].unique())
        display(len(test[np.isnan(test["sequenceNo"])]["StockID"].unique()))
        display(test[np.isnan(test["sequenceNo"])]["StockID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["IOPV"])])
        display(n2-n1)
        print((n2-n1)/n1)
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    print('----------------------------------------------------------------')
    print('SH index data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_52_03_day_pcap\\mdIndexPcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    index = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    index["StockID"] = index["ID"] - 1000000
    index = index.rename(columns={"open":"openPrice"})

    readPath = 'A:\\KR_daily_data\\' + y + '\\SH\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs == 16) | (dateLs == 300) | (dateLs == 852) | (dateLs == 905)]
    logSH = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        df = pd.read_csv(i)
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        logSH += [df]
    logSH = pd.concat(logSH).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    for i in range(1, 6):
        if i == 1:
            logSH["bid" + str(i) + "p"] = logSH["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSH["ask" + str(i) + "p"] = logSH["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSH["bid" + str(i) + "q"] = logSH["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            logSH["ask" + str(i) + "q"] = logSH["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            logSH["bid" + str(i) + "p"] = logSH["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSH["ask" + str(i) + "p"] = logSH["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSH["bid" + str(i) + "q"] = logSH["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            logSH["ask" + str(i) + "q"] = logSH["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    logSH = logSH.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    logSH["time"] = (logSH["SendingTime"] - int(y)*1000000000).astype(np.int64)
    
    in_dex = [16, 300, 852, 905]
    index = index[index["StockID"].isin(in_dex)]
    print(index["StockID"].unique())
    
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["time"] >= 91500000) & (index["time"] <= 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    for cols in ["cum_amount", "close", "openPrice"]:
        data1_1[cols] = (data1_1[cols]*10000).round(0)
        
        
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["IOPV"].count()
    n2 = test["sequenceNo"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["sequenceNo"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["IOPV"].count()
    n2 = test["sequenceNo"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["IOPV"])])
    
    del index
    del logSH
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdL2Pcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    logSZ1 = logSZ1.loc[:, ["clockAtArrival", "sequenceNo", "ID", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "open", "cum_tradesCnt"]]
    logSZ1 = logSZ1.rename(columns={"open":"openPrice", "cum_tradesCnt":"numTrades"})
    logSZ1["StockID"] = logSZ1["ID"] - 2000000

    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    logSZ = []
    ll = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        logSZ += [df]
    del df
    logSZ = pd.concat(logSZ).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    for i in range(1, 6):
        if i == 1:
            logSZ["bid" + str(i) + "p"] = logSZ["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSZ["ask" + str(i) + "p"] = logSZ["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSZ["bid" + str(i) + "q"] = logSZ["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            logSZ["ask" + str(i) + "q"] = logSZ["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            logSZ["bid" + str(i) + "p"] = logSZ["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSZ["ask" + str(i) + "p"] = logSZ["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSZ["bid" + str(i) + "q"] = logSZ["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            logSZ["ask" + str(i) + "q"] = logSZ["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    logSZ = logSZ.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    logSZ["time"] = (logSZ["QuotTime"] - int(y)*1000000000).astype(np.int64)
    print(datetime.datetime.now() - startTm)    
    
    
    startTm = datetime.datetime.now()
    data1 = logSZ[(logSZ["time"] >= 91500000) & (logSZ["time"] < 150000000)]
    data2 = logSZ1[(logSZ1["time"] >= 91500000) & (logSZ1["time"] < 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))
   
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    for i in ["cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "openPrice"]:
        data1_1[i] = (data1_1[i] * 10000).round(0)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    print(datetime.datetime.now() - startTm)
    n1 = test["ImageStatus"].count()
    n2 = test["sequenceNo"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["sequenceNo"])])
        print(len(test[np.isnan(test["sequenceNo"])])/n1)
        print(np.sort(test[np.isnan(test["sequenceNo"])]["time"].unique()))
        print(len(np.sort(test[np.isnan(test["sequenceNo"])]["StockID"].unique())))
        print(np.sort(test[np.isnan(test["sequenceNo"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["ImageStatus"])])
        display(n2-n1)
    del logSZ
    del logSZ1
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdOrderPcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    OrderLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    OrderLogSZ1["SecurityID"] = OrderLogSZ1["ID"] - 2000000
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"time":'TransactTime'})
    

    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\order\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    OrderLogSZ = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i, encoding='GBK')
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        OrderLogSZ += [df]
    OrderLogSZ = pd.concat(OrderLogSZ).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    OrderLogSZ = OrderLogSZ.rename(columns={"OrdType": "OrderType"})
    OrderLogSZ["Price"] = OrderLogSZ["Price"]*10000
    OrderLogSZ["Price"] = OrderLogSZ["Price"].round(0)
    OrderLogSZ["TransactTime"] = (OrderLogSZ["TransactTime"] - int(y) * 1000000000).astype(np.int64)
#     OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["Side"] != 'F']
    display(OrderLogSZ["Side"].unique())
    display(OrderLogSZ["ChannelNo"].unique())
    OrderLogSZ["Side"] = np.where(OrderLogSZ["Side"] == '1', 1, np.where(
    OrderLogSZ["Side"] == '2', 2, OrderLogSZ["Side"]))
    display(OrderLogSZ[((OrderLogSZ["Side"] != 1) & (OrderLogSZ["Side"] != 2)) | (OrderLogSZ["OrderType"].isnull())])
    OrderLogSZ["OrderType"] = np.where(OrderLogSZ["OrderType"] == 2, '2', np.where(
        OrderLogSZ["OrderType"] == 1, '1', OrderLogSZ['OrderType']))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    

    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique())))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime", "Side", 'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["SendingTime"].count()
    n2 = ree["sequenceNo"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["sequenceNo"])])
        print(len(ree[np.isnan(ree["sequenceNo"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["SendingTime"])])
        print(np.sort(ree[np.isnan(ree["SendingTime"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["SendingTime"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["SendingTime"])]["SecurityID"].unique())
        display(n2-n1)
    del OrderLogSZ
    del OrderLogSZ1
    del ree
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zt_52_03_day_pcap\\mdTradePcap_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    SH1["SecurityID"] = SH1["ID"] - 1000000
    SH1 = SH1.rename(columns={"time":'TransactTime'})

    readPath = 'A:\\KR_daily_data\\' + y + '\\SH\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    SH = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        SH += [df]
    SH = pd.concat(SH).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    SH["TransactTime"] = (SH["TradeTime"] - int(y) * 1000000000).astype(np.int64)
    SH["TradePrice"] = SH["TradePrice"] * 10000
    SH["TradePrice"] = SH["TradePrice"].round(0)
    SH["TradeMoney"] = SH["TradeAmount"] * 10000
    SH["TradeMoney"] = SH["TradeMoney"].round(0)
    SH["ExecType"] = 'F'
    SH = SH.rename(columns={"TradeIndex":"ApplSeqNum", "BuyNo":"BidApplSeqNum", "SellNo":"OfferApplSeqNum"})
    
    display(len(SH["SecurityID"].unique()))
    display(len(SH1["SecurityID"].unique()))
    display(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    display(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["TradeTime"].count()
    n2 = re["sequenceNo"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(len(re[np.isnan(re["sequenceNo"])]))
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["TradeTime"])])
        print(np.sort(re[np.isnan(re["TradeTime"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["TradeTime"])]["SecurityID"].unique()))
        print(re[np.isnan(re["TradeTime"])]["SecurityID"].unique())
        display(n2-n1)
    del SH
    del SH1
    del re
    
    
    
    
    
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdTradePcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    TradeLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["ID"] - 2000000
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"time":'TransactTime'})
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    TradeLogSZ = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        TradeLogSZ += [df]
    TradeLogSZ = pd.concat(TradeLogSZ).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    TradeLogSZ["TransactTime"] = (TradeLogSZ["TransactTime"] - int(y) * 1000000000).astype(np.int64)
    TradeLogSZ["TradePrice"] = TradeLogSZ["Price"] * 10000
    TradeLogSZ["TradePrice"] = TradeLogSZ["TradePrice"].round(0)
    TradeLogSZ = TradeLogSZ.rename(columns={"Qty":"TradeQty"})
    TradeLogSZ["TradeMoney"] = (TradeLogSZ["TradePrice"] * TradeLogSZ["TradeQty"]).astype(np.int64)
    TradeLogSZ["TradeBSFlag"] = 'N'
    
    
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique())))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()))


    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["Price"].count()
    n2 = re["sequenceNo"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(len(re[np.isnan(re["sequenceNo"])]))
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["Price"])])
        print(np.sort(re[np.isnan(re["Price"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["Price"])]["SecurityID"].unique()))
        print(re[np.isnan(re["Price"])]["SecurityID"].unique())
        display(n2-n1)
    del TradeLogSZ
    del TradeLogSZ1
    del re

----------------------------------------------------------------
20200708
----------------------------------------------------------------
SH index data:
0:00:01.286764


ValueError: No objects to concatenate

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200702']:
    print('----------------------------------------------------------------')
    
    readPath = '\\\\mentos\\dailyRawData\\logs_' + y + '_zs_96_03_day_pcap\\mdTradePcap_SZ_***'
    dataPathLs = np.array(glob.glob(readPath))
    
    startTm = datetime.datetime.now()
    TradeLogSZ1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)
    
    TradeLogSZ1["SecurityID"] = TradeLogSZ1["ID"] - 2000000
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"time":'TransactTime'})
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    TradeLogSZ = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        TradeLogSZ += [df]
    TradeLogSZ = pd.concat(TradeLogSZ).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    TradeLogSZ["TransactTime"] = (TradeLogSZ["TransactTime"] - int(y) * 1000000000).astype(np.int64)
    TradeLogSZ["TradePrice"] = TradeLogSZ["Price"] * 10000
    TradeLogSZ["TradePrice"] = TradeLogSZ["TradePrice"].round(0)
    TradeLogSZ = TradeLogSZ.rename(columns={"Qty":"TradeQty"})
    TradeLogSZ["TradeMoney"] = (TradeLogSZ["TradePrice"] * TradeLogSZ["TradeQty"]).astype(np.int64)
    TradeLogSZ["TradeBSFlag"] = 'N'
    
    
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique())))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()))


    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["Price"].count()
    n2 = re["sequenceNo"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(len(re[np.isnan(re["sequenceNo"])]))
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["Price"])])
        print(np.sort(re[np.isnan(re["Price"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["Price"])]["SecurityID"].unique()))
        print(re[np.isnan(re["Price"])]["SecurityID"].unique())
        display(n2-n1)

----------------------------------------------------------------
0:01:22.397937
0:02:35.778328


2221

3056

0

set()

2221
2221
Index(['clockAtArrival', 'sequenceNo', 'ID', 'TransactTime', 'ApplSeqNum',
       'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum',
       'OfferApplSeqNum', 'SecurityID', 'TradeBSFlag'],
      dtype='object')
----------------------------------------------------------------
SZ trade data:
43709827
38597543
43709827
-----------------------------------------------


'test is not complete:'

,ApplSeqNum,BidApplSeqNum,SendingTime,Price,ChannelNo,TradeQty,OfferApplSeqNum,Amt,ExecType,TransactTime,SecurityID,TradePrice,TradeMoney,TradeBSFlag,clockAtArrival,sequenceNo,ID
2210,20962885,0,20200702143350000,0.00,2012,1000,4847976,0.0,4,143350800,300313,0.0,0,N,NaN,NaN,NaN
2211,20971779,19408697,20200702143356000,0.00,2012,500,0,0.0,4,143356760,300313,0.0,0,N,NaN,NaN,NaN
2212,20992901,20198727,20200702143411000,0.00,2012,2200,0,0.0,4,143411150,300313,0.0,0,N,NaN,NaN,NaN
2213,21002653,21002652,20200702143417000,4.75,2012,100,10214695,0.0,F,143417410,300313,47500.0,4750000,N,NaN,NaN,NaN
2214,21008602,21008601,20200702143421000,4.75,2012,2200,10214695,0.0,F,143421440,300313,47500.0,104500000,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43709822,22811845,22745253,20200702150006000,15.18,2011,117,22669250,0.0,F,150000000,300349,151800.0,17760600,N,NaN,NaN,NaN
43709823,22811846,22668938,20200702150006000,15.18,2011,883,22669250,0.0,F,150000000,300349,151800.0,134039400,N,NaN,NaN,NaN
43709824,22811847,22668938,20200702150006000,15.18,2011,317,22678917,0.0,F,150000000,300349,151800.0,48120600,N,NaN,NaN,NaN
43709825,22811848,22722386,20200702150006000,15.18,2011,30000,22678917,0.0,F,150000000,300349,151800.0,4554000000,N,NaN,NaN,NaN


5112284
[ 95809550  95809560 103333930 ... 145659980 145659990 150000000]
2221
[300313    768    407 ...    703 300088 300349]


In [6]:
TradeLogSZ1.groupby("SecurityID")['TransactTime'].max().min()

143105290

In [5]:
np.sort(re[np.isnan(re["sequenceNo"]) & (re["TransactTime"] < 143350060)]["TransactTime"].unique())

array([ 95809550,  95809560, 103333930, 105106010, 133549230, 133554320,
       135204400, 135403170], dtype=int64)

In [8]:
re[np.isnan(re["sequenceNo"]) & (re["TransactTime"] < 143350060) & (re['ChannelNo'] != 4001)]

,ApplSeqNum,BidApplSeqNum,SendingTime,Price,ChannelNo,TradeQty,OfferApplSeqNum,Amt,ExecType,TransactTime,SecurityID,TradePrice,TradeMoney,TradeBSFlag,clockAtArrival,sequenceNo,ID
599038,6024605,5906012,20200702095809000,7.19,2013,200,6024604,0.0,F,95809550,426,71900.0,14380000,N,NaN,NaN,NaN
599039,6024606,0,20200702095809000,0.00,2013,100,6003001,0.0,4,95809560,426,0.0,0,N,NaN,NaN,NaN
599040,6024611,0,20200702095809000,0.00,2013,700,6002991,0.0,4,95809560,426,0.0,0,N,NaN,NaN,NaN
599041,6024612,0,20200702095809000,0.00,2013,200,6002992,0.0,4,95809560,426,0.0,0,N,NaN,NaN,NaN


In [12]:
TradeLogSZ[(TradeLogSZ["SecurityID"] == 426) & (TradeLogSZ["ApplSeqNum"] >= 6024605)]

,ApplSeqNum,BidApplSeqNum,SendingTime,Price,ChannelNo,TradeQty,OfferApplSeqNum,Amt,ExecType,TransactTime,SecurityID,TradePrice,TradeMoney,TradeBSFlag
599038,6024605,5906012,20200702095809000,7.19,2013,200,6024604,0.0,F,95809550,426,71900.0,14380000,N
599039,6024606,0,20200702095809000,0.00,2013,100,6003001,0.0,4,95809560,426,0.0,0,N
599040,6024611,0,20200702095809000,0.00,2013,700,6002991,0.0,4,95809560,426,0.0,0,N
599041,6024612,0,20200702095809000,0.00,2013,200,6002992,0.0,4,95809560,426,0.0,0,N
599042,6024614,5906012,20200702095809000,7.19,2013,100,6024613,0.0,F,95809560,426,71900.0,7190000,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636836,23418782,23316903,20200702150001000,7.45,2013,3700,23396555,0.0,F,150000000,426,74500.0,275650000,N
636837,23418783,23318402,20200702150001000,7.45,2013,600,23396555,0.0,F,150000000,426,74500.0,44700000,N
636838,23418784,23318466,20200702150001000,7.45,2013,3700,23396555,0.0,F,150000000,426,74500.0,275650000,N
636839,23418785,23318466,20200702150001000,7.45,2013,3000,23400890,0.0,F,150000000,426,74500.0,223500000,N


In [11]:
TradeLogSZ1[(TradeLogSZ1["SecurityID"] == 426) & (TradeLogSZ1["ApplSeqNum"] >= 6024605)]

,clockAtArrival,sequenceNo,ID,TransactTime,ApplSeqNum,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,SecurityID,TradeBSFlag
11100833,1593655098682796,56539753,2000426,95809560,6024614,F,71900,100,7190000,5906012,6024613,426,N
11100834,1593655098682796,56539755,2000426,95809560,6024616,F,71900,300,21570000,5906012,6024615,426,N
11100835,1593655098682796,56539756,2000426,95809560,6024617,F,71900,500,35950000,5909325,6024615,426,N
11100836,1593655098682796,56539758,2000426,95809560,6024619,F,71900,300,21570000,5909325,6024618,426,N
11100851,1593655098683990,56539782,2000426,95809560,6024627,4,0,400,0,0,6003002,426,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42135727,1593671636040222,212031534,2000426,143346870,20908529,F,75100,5200,390520000,20908528,20892599,426,N
42136362,1593671636230660,212034120,2000426,143347060,20908804,F,75100,1600,120160000,20908803,20892599,426,N
42138455,1593671636880716,212042635,2000426,143347710,20909884,F,75100,5200,390520000,20909883,20892599,426,N
42140404,1593671637477824,212050502,2000426,143348300,20910802,4,0,2100,0,0,20671616,426,N
